In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [ ]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(32, activation='relu')) #隨便想一個數字

In [ ]:
model.add(Dense(10, activation='softmax')) #不可以更改10

#### 看一下我們的神經網路

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

#### 組裝

In [ ]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=20) #epochs訓練次數

Epoch 1/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0062 - accuracy: 0.9605
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0058 - accuracy: 0.9633
Epoch 3/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0055 - accuracy: 0.9653
Epoch 4/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0052 - accuracy: 0.9668
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0050 - accuracy: 0.9685
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0047 - accuracy: 0.9700
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0045 - accuracy: 0.9707
Epoch 8/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0044 - accuracy: 0.9723
Epoch 9/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0042 - accuracy: 0.9730
Epoch 10/20
469/469 [==============================] - 2s 3ms/step - loss: 0.0041 - accuracy: 0.9744

### Step 3. 預測

In [ ]:
result = model.predict_classes(x_test)

### 看看測試資料表現如何

In [ ]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0031 - accuracy: 0.9802


In [ ]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 98.02%


In [ ]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [ ]:
from ipywidgets import interact_manual

In [ ]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [ ]:
model.save('myCNNmodel.h5')